In [3]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics.scorer import make_scorer
from datetime import datetime

%matplotlib inline

NameError: name 'data_all' is not defined

In [5]:
data_train = pd.read_csv(r"C:/Users/achut/Downloads/nagesh/Counterfeit_train.csv")

In [6]:
data_test = pd.read_csv(r"C:/Users/achut/Downloads/nagesh/Counterfeit_test.csv")

In [7]:
data_test['Counterfeit_Sales'] = np.nan
data_train['data'] = 'train' 
data_test['data'] = 'test'
data_test = data_test[data_train.columns]
data_all = pd.concat([data_train,data_test],axis=0)

# Filling missing values

In [8]:
data_all['Counterfeit_Weight'].fillna(data_all['Counterfeit_Weight'].mean(), inplace=True)

# Creating Dummies

In [9]:
##CReating dummies for categorical vars
def dummies(data,var,freq_cutoff=0):
    t= data[var].value_counts(normalize=True) 
    t=t[t.values>freq_cutoff] 
    t=t.sort_values() 
    t_min=t.idxmin()
    t=t.drop([t_min])
    categories=t.index

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name)
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var])
    return data

In [10]:
datetime.now().year-data_all['Active_Since']

0       25
1       37
2       25
3       25
4       37
        ..
1700    37
1701    18
1702    15
1703    13
1704    20
Name: Active_Since, Length: 8523, dtype: int64

In [11]:
data_all['no_of_years_active']=datetime.now().year-data_all['Active_Since']
data_all=data_all.drop(["Active_Since"],1)

In [12]:
data_all = dummies(data_all,'Area_City_Type')
data_all = dummies(data_all,'Area_Type')
data_all = dummies(data_all,'Area_dist_level')
data_all = dummies(data_all,'Medicine_Type',0.05)
data_all = dummies(data_all,'SidEffect_Level')
#maine sirf yahn tk kia h

In [13]:
data_all.dtypes

Medicine_ID                          object
Counterfeit_Weight                  float64
DistArea_ID                          object
Medicine_MRP                        float64
Availability_rating                 float64
Counterfeit_Sales                   float64
data                                 object
no_of_years_active                    int64
Area_City_Type_Tier2                  int32
Area_City_Type_Tier3                  int32
Area_Type_CityLimits                  int32
Area_Type_MidTownResidential          int32
Area_Type_DownTown                    int32
Area_dist_level_Small                 int32
Area_dist_level_Unknown               int32
Area_dist_level_Medium                int32
Medicine_Type_Tranquilizers           int32
Medicine_Type_Mstablizers             int32
Medicine_Type_Cardiac                 int32
Medicine_Type_Antipyretics            int32
Medicine_Type_OralContraceptives      int32
Medicine_Type_Antiseptics             int32
Medicine_Type_Hreplacements     

In [14]:
garbage_cols=["Medicine_ID","DistArea_ID"]
data_all.drop(columns=garbage_cols,axis=1,inplace=True)

In [15]:
data_all.head()

,Counterfeit_Weight,Medicine_MRP,Availability_rating,Counterfeit_Sales,data,no_of_years_active,Area_City_Type_Tier2,Area_City_Type_Tier3,Area_Type_CityLimits,Area_Type_MidTownResidential,...,Area_dist_level_Medium,Medicine_Type_Tranquilizers,Medicine_Type_Mstablizers,Medicine_Type_Cardiac,Medicine_Type_Antipyretics,Medicine_Type_OralContraceptives,Medicine_Type_Antiseptics,Medicine_Type_Hreplacements,Medicine_Type_Antibiotics,SidEffect_Level_mild
0,13.100000,160.2366,0.070422,1775.5026,train,25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14.157645,110.4384,0.013000,3069.1520,train,37,0,1,1,0,...,1,0,1,0,0,0,0,0,0,1
2,9.025000,259.4092,0.060783,2603.0920,train,25,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,11.800000,99.9830,0.065555,1101.7130,train,25,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,14.157645,56.4402,0.248859,158.9402,train,37,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


# splitting back in train and test 

In [16]:
data_train=data_all[data_all['data']=='train'] 
del data_train['data'] 
data_test=data_all[data_all['data']=='test']
data_test.drop(['Counterfeit_Sales','data'],axis=1,inplace=True) 

C:\Users\achut\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# splitting train in train1 and train2 

In [17]:
train1, train2 = train_test_split(data_train, test_size = 0.25,random_state=2)

x_train1=train1.drop(["Counterfeit_Sales"],1)
y_train1=train1["Counterfeit_Sales"]

x_train2=train2.drop(["Counterfeit_Sales"],1)
y_train2=train2["Counterfeit_Sales"]

x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

## Random Forest Regressor 

In [18]:
#function make_scorer converts this to a function which we can pass to GridSearchCV/RandomSearchCV
def loss_func(x, y):return((np.abs(x-y)).mean())
    
my_scorer = make_scorer(loss_func, greater_is_better=False)

In [19]:
rg = RandomForestRegressor(n_jobs=-1,verbose=1)

        
param_dist = {"n_estimators":[100,300,500,700],
              'min_samples_split':[5,7,9],
               'bootstrap':[True],
                'n_jobs':[-1],
               'max_depth':[5,10,20],
               'max_features':[5,10,20],
               'min_samples_leaf':[6,8,10]}

n_iter_search = 250
random_search = RandomizedSearchCV(rg, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   scoring=my_scorer,
                                   cv=10,
                                   random_state=2)

In [20]:
random_search.fit(x_train1, y_train1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | ela

[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s f

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0

[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 700 out of 700 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | ela

...

[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 493 out of 500 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 

[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 tasks 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 235 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 235 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend L

RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(n_jobs=-1, verbose=1),
                   n_iter=250,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [5, 10, 20],
                                        'max_features': [5, 10, 20],
                                        'min_samples_leaf': [6, 8, 10],
                                        'min_samples_split': [5, 7, 9],
                                        'n_estimators': [100, 300, 500, 700],
                                        'n_jobs': [-1]},
                   random_state=2,
                   scoring=make_scorer(loss_func, greater_is_better=False))

In [21]:
print(random_search.best_estimator_)

RandomForestRegressor(max_depth=10, max_features=10, min_samples_leaf=10,
                      min_samples_split=7, n_jobs=-1, verbose=1)


In [22]:
rg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features=20, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=7,
                      min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=1,
                      warm_start=False)

In [23]:
rg.fit(x_train1, y_train1)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.8s finished


RandomForestRegressor(max_depth=5, max_features=20, min_samples_leaf=10,
                      min_samples_split=7, n_estimators=300, n_jobs=-1,
                      verbose=1)

In [24]:
predicted_train1=rg.predict(x_train1)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


In [25]:
predicted_train2=rg.predict(x_train2)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


In [27]:
residual=predicted_train2-y_train2
residual

5443    -228.731243
4972     224.041001
3383      98.554541
1350     228.063448
145     -171.462227
           ...     
3682      60.512375
436     1944.620469
4588    -508.338999
6266    1082.082721
1006     302.863575
Name: Counterfeit_Sales, Length: 1705, dtype: float64

In [30]:
rmse=np.sqrt(np.dot(residual,residual)/len(predicted_train1))
rmse

626.2461453580947

In [31]:
from sklearn.metrics import mean_absolute_error

In [32]:
mae_train1=mean_absolute_error(y_train1, predicted_train1)
mae_train2=mean_absolute_error(y_train2, predicted_train2)

In [33]:
Score1 = 1-(mae_train1/1660)
Score1

0.5581041572069818

In [34]:
Score2 = 1-(mae_train2/1660)
Score2

0.545780209091522

In [35]:
predicted_test=rg.predict(data_test)
predicted_test

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


array([2218.51804273, 4010.66277596, 1516.20194357, ..., 2972.03291892,
       3751.9195381 , 3956.65699792])

In [39]:
pd.DataFrame(predicted_test).to_csv("Marshivane_Nagesh_P3_part2.csv",index=False)